In [ ]:
import numpy as np
from src.network import NeuralNetwork

In [ ]:
def numeric_gradient(f, x, eps=1e-5):
    grad = np.zeros_like(x)
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        idx = it.multi_index
        old = x[idx]
        x[idx] = old + eps
        fx1 = f()
        x[idx] = old - eps
        fx2 = f()
        x[idx] = old
        grad[idx] = (fx1 - fx2) / (2 * eps)
        it.iternext()
    return grad


In [ ]:
np.random.seed(0)
net = NeuralNetwork(
    input_dim=3,
    layers_config=[
        {"units": 4, "activation": "sigmoid"},
        {"units": 2, "activation": "softmax"},
    ],
    loss="cross_entropy"
)

X = np.random.randn(5, 3).astype(np.float32)
y = np.array([0,1,0,1,1])
y_oh = np.zeros((5,2)); y_oh[np.arange(5), y] = 1

def f():
    return net.compute_loss_and_gradients(X, y_oh)

# forward + backward para tener gradientes analíticos
loss = f()
params_grads = net.get_params_and_grads()
max_diff = 0.0
for (p, g) in params_grads:
    def f_param():
        return net.compute_loss_and_gradients(X, y_oh)
    num_g = numeric_gradient(f_param, p)
    diff = np.max(np.abs(num_g - g))
    max_diff = max(max_diff, diff)
    print("max diff for param:", diff)

print("GLOBAL max diff:", max_diff)
